## 中标公告买卖关系提取


### 对中标数据进行预处理
* 清除white space string，包括\xa0，\t等等（表格中的无效信息，会使得提取的表格信息中多出很多无效的行）。提取表格时，若没有\n等，会缺失有效信息，因此仅清除 \xa0，\t，&nbsp等，不可清除所有空白字符


### 提取中标人信息
'中标供应商数量'用于后续区分一个供应商和多个供应商（文本形式提取的供应商数据，此字段值为-1，需要手动查看是否有多个供应商情况）


~~'中标金额'分为了三个字段：'中标金额','中标金额（万元）','总中标金额'~~需要看提取出的金额是否都有明显的单位，需要匹配“万元”，并在数字后加上

#### 表格形式
* 表格有三大元素：table、tr、td
    - 表格由table标签来定义。每个表格均有若干行（由tr标签定义），每行被分割为若干单元格（由td标签定义）。
        - 字母tr指表格行（table row）
        - 字母td指表格数据（table data），即数据单元格的内容。  
* 多个中标人的情况，按顺序放入列表中
* 确定表格列数
    * 列数 = 单元格数/行数 （部分公告有多于一个表格，不适用），判断是否整数，不是时暂定为4或7 → 多于一个表格大概率得到非整数


#### 文本形式
* 部分公告的中标人信息不是以表格的形式显示，需要利用xpath("//p/text()")的内容和正则来提取，这部分需要注意"//p/text()"是否能代表所有公告的中标人信息部分的标签
* 若存在多个中标人，暂时需要手动分离

#### 若bidder_result中没有相应字段，在tender_result中查找


### 提取采购商信息


* 采购商信息可能存在两种标签路径中"//tr/td/text()"  "//tr/tdclass/text()" （可能还有其他的）"//p/span/text()"
* 字段名称可能有不同的表达，用字典tender_info_dict统一（中标人信息（无论是表格提取还是标签提取）用winner_bidder_info_dict统一）

### 待确认
* !!!!!!!!!!!!!!!提取'//tbody/tr/td/text()'的表格时，到底需要保留哪些符号，现有的删除掉的空白符有影响，会导致提取不全
* 没有表格的公告是否至多一个中标商(若不是，result提取出的形式是什么，如何加入)
* ‘万元’与‘元’是否概括了所有金额单位的情况
* 没有单位的金额可否直接加上‘元’


In [1]:
from lxml import etree
import re
import os

In [267]:
def getWinnerBidderInfo(_html):
    announcement_info = {}
    num = len(bidder_result_table)
    print(bidder_result_table)
    # 表格的字段名称不一致，也需dict
    info_list = ['序号', '中标供应商名称', '中标供应商联系地址', '中标金额(万元)']
    info_standard_name_list = ['序号', '中标供应商名称', '中标供应商联系地址', '中标金额']
    successful_supplier_num = 0 # 初始化中标供应商数量，如果既没有表格信息，文本也提取不出来，则中标供应商数量为0
    
    # 初始化announcement_info各项为空列表，即使只有一个中标供应商，也存到列表中
    for i in info_standard_name_list[1:]:
        announcement_info[i] = []
    
    
    if num == 0:
        """
        一些公告中，中标人的信息不是以表格的形式展示的，此时需要根据标签尝试提取
        没有表格的，提取出后bidder_result_1可能包含如下形式
         '中标供应商名称、联系地址及中标金额：',
         '中标人名称:标新科技（北京）有限公司',
         '地址:北京市昌平区科技园区永安路36号',
         '中标金额： 2,415,000.00元'
        """
        for winner_bidder_info in winner_bidder_info_dict:
            # 若匹配到过则跳出，否则会多出字段，e.g.同时有“总中标金额”和“中标金额”→ 没有表格时假设至多一个中标商的情况
            # 先清洗一下空白字符,否则若匹配到空字符串""，则列表不为空，但没有有效内容 → 在每次插入时清洗
            if announcement_info[winner_bidder_info_dict[winner_bidder_info]]:
                #announcement_info[winner_bidder_info_dict[winner_bidder_info]] = re.sub(r'\s*',"",announcement_info[winner_bidder_info_dict[winner_bidder_info]])
                continue
            bidder_result_var = 0
            bidder_result_var_end = 1
            for bidder_result_var in range(bidder_result_var_end + 1):
                bidder_result_cur = eval('bidder_result_{}'.format(bidder_result_var))
                for info in bidder_result_cur:
                    
                    # 匹配字段名称信息后，后面可能有冒号、空格等符号，需要一起替换掉，但数字不可替换，金额处会用到，不可直接[^\u4e00-\u9fa5]匹配非汉字
                    # 匹配时需直接匹配冒号，否则可能将下例包含进来：'中标供应商名称、联系地址及中标金额：'
                    print(bidder_result_var, info)
                    if  re.search( r'^{}[:：]+\s*'.format(winner_bidder_info), info):
                        bidder_info = re.sub(r'^{}[:：]+\s*'.format(winner_bidder_info), "", info)
                        # 先清洗一下空白字符,否则若匹配到空字符串""，则列表不为空，但没有有效内容 → 在每次插入时清洗
                        bidder_info = re.sub(r'\s*','',bidder_info)
                        if bidder_info:
                            if re.search(r'万元', info): # 将匹配“万元”的单位加到数字后面
                                announcement_info[winner_bidder_info_dict[winner_bidder_info]].append(+'万元')
                            else:
                                announcement_info[winner_bidder_info_dict[winner_bidder_info]].append(re.sub(r'^{}[:：]+\s*'.format(winner_bidder_info), "", info))
                    # 若仅匹配到字段名称，而没有“冒号”等，表示可能字段的内容在列表中下一个位置，而不是和字段名在一个字符串中
                    # 需注意，应匹配到字符串最后，否则类似“中标商名称、联系地址”等也会匹配到  '没有冒号的“中标供应商”''
                    elif  re.search( r'^{}\s*$'.format(winner_bidder_info), info):
                        bidder_info = bidder_result_cur[bidder_result_cur.index(info)+1]
                        # 先清洗一下空白字符,否则若匹配到空字符串""，则列表不为空，但没有有效内容 → 在每次插入时清洗
                        bidder_info = re.sub(r'\s*','',bidder_info)
                        if bidder_info:
                            if re.search(r'万元', info): # 将匹配“万元”的单位加到数字后面
                                announcement_info[winner_bidder_info_dict[winner_bidder_info]].append(bidder_info+'万元')
                            else:
                                announcement_info[winner_bidder_info_dict[winner_bidder_info]].append(bidder_info)
            # 若在 bidder_result 中没有匹配到，在 tender_result 中尝试
            if not announcement_info[winner_bidder_info_dict[winner_bidder_info]]:
                # 和上部分逻辑相同，只是从 bidder_result换为 tender_result
                tender_result_var = 0
                tender_result_var_end = 1
                for tender_result_var in range(tender_result_var_end + 1):
                    tender_result_cur = eval('tender_result_{}'.format(tender_result_var))
                    for info in tender_result_cur:

                        # 匹配字段名称信息后，后面可能有冒号、空格等符号，需要一起替换掉，但数字不可替换，金额处会用到，不可直接[^\u4e00-\u9fa5]匹配非汉字
                        # 匹配时需直接匹配冒号，否则可能将下例包含进来：'中标供应商名称、联系地址及中标金额：'
                        if  re.search( r'^{}[:：]+\s*'.format(winner_bidder_info), info):
                            if re.search(r'万元', info): # 将匹配“万元”的单位加到数字后面
                                announcement_info[winner_bidder_info_dict[winner_bidder_info]].append(re.sub(r'^{}[:：]+\s*'.format(winner_bidder_info), "", info)+'万元')
                            else:
                                announcement_info[winner_bidder_info_dict[winner_bidder_info]].append(re.sub(r'^{}[:：]+\s*'.format(winner_bidder_info), "", info))
                        # 若仅匹配到字段名称，而没有“冒号”等，表示可能字段的内容在列表中下一个位置，而不是和字段名在一个字符串中
                        # 需注意，应匹配到字符串最后，否则类似“中标商名称、联系地址”等也会匹配到  '没有冒号的“中标供应商”''
                        elif  re.search( r'^{}\s*$'.format(winner_bidder_info), info):
                            if re.search(r'万元', info): # 将匹配“万元”的单位加到数字后面
                                announcement_info[winner_bidder_info_dict[winner_bidder_info]].append(tender_result_cur[tender_result_cur.index(info)+1]+'万元')
                            else:
                                announcement_info[winner_bidder_info_dict[winner_bidder_info]].append(tender_result_cur[tender_result_cur.index(info)+1])
        
        # successful_supplier_num=-1 表示中标人信息不是从表格提取，可能需要手动清洗
        successful_supplier_num = -1
                
    
    else:
        successful_supplier_num = num/4 - 1 # 中标供应商的数量
        i = 1
        while 4 * i < num:  
            # html文件中每一行有4列，所以以4个单元为一组对列表进行分片读取，需注意，这种方法需要保证表格中没有空元素或者和开发约定好假如有空元素以空格为占位符
            # '序号', '中标供应商名称', '中标供应商联系地址', '中标金额(万元)'
            #first = (i - 1) * 4
            last = i * 4 

            # 存在多个中标供应商的情况，因此不可如下生成announcement_info
            '''
            for j in range(4):
                print(result_table[first+j])
                print(result_table[last+j])
                announcement_info[result_table[first+j]] = result_table[last+j]
                print(announcement_info)
            #print(result_table[first:last + 1])
            '''
            # 列数 = 单元格数/行数 （部分公告有多于一个表格，不适用）
            td = len(html_removing_whiteSpaceString.xpath('//tbody/tr/td')) # table data 单元格对象个数
            tr = len(html_removing_whiteSpaceString.xpath('//tbody/tr')) # table row 对象个数
            col_num = td/tr
            # 判断是否整数，不是时暂定为4或7 → 多于一个表格大概率得到非整数
            if col_num % 1 != 0:
                if len(bidder_result_table) % 7 == 0:
                    col_num = 7
                elif len(bidder_result_table) % 4 == 0:
                    col_num = 4
                else:
                    print('表格列数不确定')
            col_num = 4
            for j in range(1, 4): # 跳过序号
                #print(result_table[first+j])
                #print(result_table[last+j])
                # 将匹配“万元”的单位加到数字后面，表格的好像都是“万元”
                if info_standard_name_list[j]=='中标金额':
                    announcement_info[info_standard_name_list[j]].append(bidder_result_table[last+j]+"万元")
                else:
                    announcement_info[info_standard_name_list[j]].append(bidder_result_table[last+j])
                #print(announcement_info)
            #print(result_table[first:last + 1])
            i += 1
    announcement_info['中标供应商数量'] = successful_supplier_num # 用于后续将一个供应商和多个供应商的数据区分
    return announcement_info



In [268]:
bidder_info = ' '
bidder_info = re.sub(r'\s*','',bidder_info)
bidder_info

''

In [269]:
var = 0
for result in eval('bidder_result_{}'.format(var)):
    print(result)
    var += 1
    if var >=2:
        break

财政部唯一指定政府采购信息网络发布媒体 国家级政府采购专业网站
服务热线：400-810-1996


In [270]:
winner_bidder_info = '中标人信息'
info = '中标人信息:地址'
re.search( r'^{}[:：]+\s*'.format(winner_bidder_info), info)
re.sub(r'^{}[:：]+\s*'.format(winner_bidder_info), "", info)

'地址'

In [271]:
announcement_info = getWinnerBidderInfo(html_removing_whiteSpaceString)

for field in tender_info_dict:
    field_standard_name = tender_info_dict[field]
    if field in tender_result_0:
        announcement_info[field_standard_name] = tender_result_0[tender_result_0.index(field)+1]
    elif field in tender_result_1:
        announcement_info[field_standard_name] = tender_result_1[tender_result_1.index(field)+1]
    else:
        announcement_info[field_standard_name] = ''

['序号', '中标供应商名称', '中标供应商联系地址', '中标金额(万元)', '1', '中国软件与技术服务股份有限公司', '北京市昌平区昌盛路18号', '129.000000', '2', '神州数码信息系统有限公司', '北京市海淀区西北旺东路10号院（东区）18号楼3层101-307', '64.700000', '3', '税友软件集团股份有限公司', '杭州市滨江区浦沿街道南环路3738号', '21.600000']


In [272]:
tender_result_0

['采购项目名称',
 '河南省国家税务局2018年金税三期工程系统运维服务项目',
 '品目',
 '采购单位',
 '河南省国家税务局',
 '行政区域',
 '金水区',
 '公告时间',
 '2018年01月02日  12:12',
 '本项目招标公告日期',
 '2017年12月07日',
 '中标日期',
 '2017年12月29日',
 '评审专家名单',
 '洪超、刘雷、秦杰、郭磊、秦国玲',
 '总中标金额',
 '￥215.3 万元（人民币）',
 '项目联系人',
 '韩旭',
 '项目联系电话',
 '0371-55259965',
 '采购单位',
 '河南省国家税务局',
 '采购单位地址',
 '郑州市丰产路111号',
 '采购单位联系方式',
 '杨新艳 张书勤 0371-66767320',
 '代理机构名称',
 '河南省国贸招标有限公司',
 '代理机构地址',
 '河南省郑州市农业路72号国际企业中心B座三楼东',
 '代理机构联系方式',
 '韩旭 0371-55259965',
 '附件1',
 '附件2',
 '序号',
 '中标供应商名称',
 '中标供应商联系地址',
 '中标金额(万元)',
 '1',
 '中国软件与技术服务股份有限公司',
 '北京市昌平区昌盛路18号',
 '129.000000',
 '2',
 '神州数码信息系统有限公司',
 '北京市海淀区西北旺东路10号院（东区）18号楼3层101-307',
 '64.700000',
 '3',
 '税友软件集团股份有限公司',
 '杭州市滨江区浦沿街道南环路3738号',
 '21.600000']

In [273]:
tender_result_1

[]

In [274]:
bidder_result_0

['财政部唯一指定政府采购信息网络发布媒体 国家级政府采购专业网站',
 '服务热线：400-810-1996',
 '当前位置：',
 '»\n                ',
 '»',
 '»',
 ' 来源：',
 ' 【',
 '】 ',
 '服务/信息技术服务/运营服务/软件运营服务,服务/信息技术服务/运营服务/平台运营服务,服务/信息技术服务/运营服务/基础设施运营服务,服务/信息技术服务/运行维护服务/基础环境运维服务',
 '\u3000\u3000河南省国贸招标有限公司受河南省国家税务局的委托，就河南省国家税务局2018年金税三期工程系统运维服务项目项目（项目编号：GMFG17733-HNGS20171202）组织采购，评标工作已经结束，中标结果如下：',
 '项目编号：GMFG17733-HNGS20171202',
 '项目名称：河南省国家税务局2018年金税三期工程系统运维服务项目',
 '项目联系人：韩旭',
 '联系方式：0371-55259965',
 '采购单位名称：河南省国家税务局',
 '采购单位地址：郑州市丰产路111号',
 '采购单位联系方式：杨新艳 张书勤 0371-66767320',
 '1.项目概况：金税三期工程是国家电子政务“十二金”工程之一，系统运用先进税收管理理念和技术，实现了全国征管数据应用大集中，统一了全国征管数据标准、口径，内容涵盖了税收征管和行政管理的方方面面，具体包括了核心征管及应用总集成、个人所得税管理、税库银横向联网、管理决策平台、纳税服务平台、行政管理平台、外部数据交换平台、运维管理平台等多个应用系统。',
 '本次采购购买2018年金税三期工程系统运维服务， 包括金税三期工程核心征管系统运维服务,金税三期工程管理决策1包运维服务,金税三期工程管理决策2包运维服务。',
 '2.标包划分及各分包具体采购内容：本次采购共分A包、B包、C包三个分包。各分包采购内容、采购预算及服务期限详见下表：',
 '分包',
 '采购内容',
 '采购预算',
 '技术服务人数',
 '服务期限',
 'A包',
 '金税三期工程核心征管系统运维服务',
 '129.6万元',
 '6人',
 '一年',
 'B包',
 '金税三期工程管理决策1包运维服务',
 '64.8万元',
 '3人',
 '一年

In [275]:
bidder_result_1

['2018年01月02日 12:12', '打印', 'bm14000002', '11010602060068', '© 1999-']

In [276]:
bidder_result_table

['序号',
 '中标供应商名称',
 '中标供应商联系地址',
 '中标金额(万元)',
 '1',
 '中国软件与技术服务股份有限公司',
 '北京市昌平区昌盛路18号',
 '129.000000',
 '2',
 '神州数码信息系统有限公司',
 '北京市海淀区西北旺东路10号院（东区）18号楼3层101-307',
 '64.700000',
 '3',
 '税友软件集团股份有限公司',
 '杭州市滨江区浦沿街道南环路3738号',
 '21.600000']

In [277]:
announcement_info

{'中标供应商名称': ['中国软件与技术服务股份有限公司', '神州数码信息系统有限公司', '税友软件集团股份有限公司'],
 '中标供应商联系地址': ['北京市昌平区昌盛路18号',
  '北京市海淀区西北旺东路10号院（东区）18号楼3层101-307',
  '杭州市滨江区浦沿街道南环路3738号'],
 '中标金额': ['129.000000万元', '64.700000万元', '21.600000万元'],
 '中标供应商数量': 3.0,
 '采购项目名称': '河南省国家税务局2018年金税三期工程系统运维服务项目',
 '采购单位': '河南省国家税务局',
 '公告时间': '2018年01月02日  12:12',
 '招标公告时间': '2017年12月07日',
 '中标日期': '2017年12月29日'}

In [105]:
result = html.xpath("//tr/tdclass/text() ")

In [106]:
result

['采购项目名称',
 '品目',
 '采购单位',
 '行政区域',
 '本项目招标公告日期',
 '评审专家名单',
 '总中标金额',
 '项目联系人',
 '项目联系电话',
 '采购单位',
 '采购单位地址',
 '采购单位联系方式',
 '代理机构名称',
 '代理机构地址',
 '代理机构联系方式',
 '附件1']

In [56]:
procurement_institution

'粤北片区监狱医院'

In [107]:
result

['采购项目名称',
 '品目',
 '采购单位',
 '行政区域',
 '本项目招标公告日期',
 '评审专家名单',
 '总中标金额',
 '项目联系人',
 '项目联系电话',
 '采购单位',
 '采购单位地址',
 '采购单位联系方式',
 '代理机构名称',
 '代理机构地址',
 '代理机构联系方式',
 '附件1']

In [81]:
getWinnerBidderInfo(html)
'''
 \xa0 属于 latin1 （ISO/IEC_8859-1）中的扩展字符集字符，代表空白符nbsp(non-breaking space)。 latin1 字符集向下兼容 ASCII （ 0x20~0x7e ）
'''

['序号', '中标供应商名称', '中标供应商联系地址', '中标金额(万元)', '1', '广州医药有限公司', '广东省广州市大同路103号', '0.000000']
序号
1
{'序号': '1'}
中标供应商名称
广州医药有限公司
{'序号': '1', '中标供应商名称': '广州医药有限公司'}
中标供应商联系地址
广东省广州市大同路103号
{'序号': '1', '中标供应商名称': '广州医药有限公司', '中标供应商联系地址': '广东省广州市大同路103号'}
中标金额(万元)
0.000000
{'序号': '1', '中标供应商名称': '广州医药有限公司', '中标供应商联系地址': '广东省广州市大同路103号', '中标金额(万元)': '0.000000'}


'\n \xa0 属于 latin1 （ISO/IEC_8859-1）中的扩展字符集字符，代表空白符nbsp(non-breaking space)。 latin1 字符集向下兼容 ASCII （ 0x20~0x7e ）\n'

In [70]:
announcement_info

NameError: name 'announcement_info' is not defined

In [38]:
f1_removing_xa0 = "".join(f1.split()) # sep没有指定时，任意white space string都会作为separator，包括 \t，\xa0等等
html1_removing_xa0 = etree.HTML(f1_removing_xa0)
getWinnerBidderInfo(html1_removing_xa0)

In [27]:
f2 = open("../linliquan/sofy_classify/Bid_announcement/central/Service/2018.01.02国家质量监督检验检疫总局通关业务司三级等保测评中标公告.html","r",encoding="utf-8")
f2 = f2.read()
html2 = etree.HTML(f2)
getWinnerBidderInfo(html2)

In [29]:
f3 = open("../linliquan/sofy_classify/Bid_announcement/central/Service/2018.01.02国家质量监督检验检疫总局信息中心国家质量监督检验检疫总局执法督查司12365举报处置指挥系统运行维护服务项目中标公告.html","r",encoding="utf-8")
f3 = f3.read()
html3 = etree.HTML(f3)
getWinnerBidderInfo(html3)

In [32]:
f4 = open("../linliquan/sofy_classify/Bid_announcement/central/Service/2018.01.02河南省国家税务局2018年金税三期工程系统运维服务项目中标公告.html","r",encoding="utf-8")
f4 = f4.read()
html4 = etree.HTML(f4)
getWinnerBidderInfo(html4)

In [186]:
if __name__=='__main__':
    file_path="../../linliquan/sofy_classify/Bid_announcement/central/Service"   #原始中标数据文件夹路径
    path_list = os.listdir(file_path)
    
    for i,path in enumerate(path_list):
        f = open("../../linliquan/sofy_classify/Bid_announcement/central/Service/2018.01.022017年粤北片区监狱医院药品、医用耗材及检验试剂集中采购配送商遴选（重招）中标公告.html","r",encoding="utf-8") #读取文件
        #f = open("../../linliquan/sofy_classify/Bid_announcement/central/Service/2018.01.02国家质量监督检验检疫总局通关业务司三级等保测评中标公告.html","r",encoding="utf-8")
        #f = open("../../linliquan/sofy_classify/Bid_announcement/central/Service/2018.01.02国家质量监督检验检疫总局信息中心国家质量监督检验检疫总局执法督查司12365举报处置指挥系统运行维护服务项目中标公告.html","r",encoding="utf-8")
        #f = open("../../linliquan/sofy_classify/Bid_announcement/central/Service/2018.01.02河南省国家税务局2018年金税三期工程系统运维服务项目中标公告.html","r",encoding="utf-8")

        f = f.read()#把文件内容转化为字符串
        '''
        f_removing_whiteSpaceString = "".join(f.split()) 
        sep没有指定时，任意white space string都会作为separator，包括 \t，\xa0(即&nbsp)等等
        移除所有空白字符，会导致html解析少很多东西，故只移除\t\t\t和\xa0
        '''
        f_removing_whiteSpaceString = "".join(f.split('\t')) 
        f_removing_whiteSpaceString = "".join(f_removing_whiteSpaceString.split('&nbsp; '))
        f_removing_whiteSpaceString = "".join(f_removing_whiteSpaceString.split('&nbsp;'))
        f_removing_whiteSpaceString = "".join(f_removing_whiteSpaceString.split('\xa0'))
        # 字段名称可能有不同的表达，用字典统一
        tender_info_dict = {'采购项目名称':'采购项目名称', '采购单位':'采购单位', '公告时间':'公告时间', '招标公告时间':'招标公告时间','本项目招标公告日期':'招标公告时间', '中标日期':'中标日期'}
        winner_bidder_info_dict = {'中标人名称': '中标供应商名称','中标供应商名称': '中标供应商名称','中标商名称': '中标供应商名称', '地址':'中标供应商联系地址','中标供应商联系地址':'中标供应商联系地址','中标商地址':'中标供应商联系地址', '中标金额':'中标金额','中标金额(万元)':'中标金额', '总中标金额':'中标金额'}

        html_removing_whiteSpaceString = etree.HTML(f_removing_whiteSpaceString) #把字符串转化为可处理的格式
        html = etree.HTML(f)
        # 采购商信息标签路径 tender_result_0; tender_result_1
        tender_result_0 = html_removing_whiteSpaceString.xpath("//tr/td/text()")
        tender_result_1 = html_removing_whiteSpaceString.xpath("//tr/tdclass/text()")
        # 中标商信息标签路径 bidder_result_0; bidder_result_1
        bidder_result_0 = html_removing_whiteSpaceString.xpath("//p/text()")
        bidder_result_1 = html_removing_whiteSpaceString.xpath("//p/span/text()")
        bidder_result_table = html_removing_whiteSpaceString.xpath('//tbody/tr/td/text()') #把html文件中所有表格数据都存入result中，需注意result是一个列表

In [187]:
title = html_removing_whiteSpaceString.xpath("//title/text()")

In [190]:
html_removing_whiteSpaceString.xpath('//meta[@name="ArticleTitle"]/@content')

[]

In [188]:
title

['2017年粤北片区监狱医院药品、医用耗材及检验试剂集中采购配送商遴选（重招）中标公告']

In [69]:
import pandas as pd
f = open("../../linliquan/sofy_classify/Bid_announcement/central/Service/2018.01.022017年粤北片区监狱医院药品、医用耗材及检验试剂集中采购配送商遴选（重招）中标公告.html","r",encoding="utf-8")
f1 = open("../../linliquan/sofy_classify/Bid_announcement/central/Service/2018.01.02国家质量监督检验检疫总局通关业务司三级等保测评中标公告.html","r",encoding="utf-8")
f2 = open("../../linliquan/sofy_classify/Bid_announcement/central/Service/2018.01.02国家质量监督检验检疫总局信息中心国家质量监督检验检疫总局执法督查司12365举报处置指挥系统运行维护服务项目中标公告.html","r",encoding="utf-8")
f3 = open("../../linliquan/sofy_classify/Bid_announcement/central/Service/2018.01.02河南省国家税务局2018年金税三期工程系统运维服务项目中标公告.html","r",encoding="utf-8")
pd.read_html(f)[1]

,0,1,2,3
0,序号,中标供应商名称,中标供应商联系地址,中标金额(万元)
1,1,广州医药有限公司,广东省广州市大同路103号,0.000000


In [280]:
file_path="../linliquan/sofy_classify/Bid_announcement/central/Service"   #原始中标数据文件夹路径
path_list = os.listdir(file_path)
for i,path in enumerate(path_list):
    print(i,path)

0 2018.01.022017年粤北片区监狱医院药品、医用耗材及检验试剂集中采购配送商遴选（重招）中标公告.html
1 2018.01.02中国人民武装警察部队总医院保安（含停车管理）服务项目中标公告.html
2 2018.01.02中国农业电影电视中心《食尚大转盘》栏目舞美灯光大屏幕设计制作采购项目中标公告.html
3 2018.01.02中国农业电影电视中心农视网三级等保网络安全设备与系统集成采购项目中标公告.html
4 2018.01.02中国文化传媒集团信息化技术支撑平台建设项目（一期）中标公告.html
5 2018.01.02中国科学院高能物理研究所高海拔宇宙线观测站（LHAASO）探测器与设备运输服务项目（第二次）中标公告.html
6 2018.01.02中央电视台涉央视舆情监测处置分析服务项目中标公告.html
7 2018.01.02中央电视台租赁节目制播用发电机供应商入围项目中标公告.html
8 2018.01.02中央财经大学修缮工程类采购代理机构引入项目中标公告.html
9 2018.01.02中央财经大学货物、服务类采购代理机构引入项目中标公告.html
10 2018.01.02全国妇联维权服务信息管理软件项目合同公告.html
11 2018.01.02商务部国际经济合作事务局援某国友好医院二期项目采购公告中标公告.html
12 2018.01.02国家税务总局个人所得税微观模拟模型项目中标公告.html
13 2018.01.02国家质量监督检验检疫总局信息中心国家质量监督检验检疫总局执法督查司12365举报处置指挥系统运行维护服务项目中标公告.html
14 2018.01.02国家质量监督检验检疫总局通关业务司三级等保测评中标公告.html
15 2018.01.02河南省国家税务局2018年金税三期工程系统运维服务项目中标公告.html
16 2018.01.032018年度《中国注册会计师》杂志印制邮寄服务项目中标公告.html
17 2018.01.03中华人民共和国上海出入境检验检疫局危险化学品安全数据单（SDS）审核业务项目中标公告.html
18 2018.01.03中华人民共和国黄埔海关黄埔海关智能物流监控平台项目中标公告.html
19 2018.01.03中国经济信息社有限公司金融信息平台一期2

792 2018.04.23国家卫星气象中心风云三号（02批）气象卫星地面应用系统工程监测分析服务系统Ⅱ期业务应用软件（大气环境监测研究和应用系统）研制中标公告.html
793 2018.04.23国家行政学院2018年网络和信息系统运行维护服务外包中标公告.html
794 2018.04.23环境保护部信息中心生态环境保护信息化工程项目（生态环境部建设部分）监理服务中标公告.html
795 2018.04.23环境保护部信息中心生态环境保护信息化工程项目（生态环境部建设部分）系统总集成中标公告.html
796 2018.04.23环境保护部信息中心生态环境大数据2018年度建设项目-国家排污许可管理信息平台（二期）中标公告.html
797 2018.04.23环境保护部卫星环境应用中心环境卫星监测及航空遥感能力建设—国家生态保护红线监管平台项目信息系统建设第四包：信息系统标准规范中标公告.html
798 2018.04.23西安市税务干部学校后勤服务招标项目中标公告.html
799 2018.04.24上海海关学院统一身份认证平台升级项目中标公告.html
800 2018.04.24中国国土资源航空物探遥感中心2018年Worldview及Pleiades等卫星数据采购中标公告.html
801 2018.04.24中国国土资源航空物探遥感中心2018年航空物探勘查飞行任务承担项目（南海中南部航空物探调查项目)中标公告.html
802 2018.04.24中国国土资源航空物探遥感中心三峡库区2018年度地面控制点测量及空三作业中标公告.html
803 2018.04.24中国国土资源航空物探遥感中心优于0.5m立体相对遥感数据采购中标公告.html
804 2018.04.24中国国土资源航空物探遥感中心俄罗斯南部地区及阿曼等国资源环境卫星遥感解译中标公告.html
805 2018.04.24中国国土资源航空物探遥感中心海洋地球物理数据综合处理算法模型与集群运算技术研究中标公告.html
806 2018.04.24中国国土资源航空物探遥感中心海洋地质与地球物理专题数据库整合中标公告.html
807 2018.04.24中国国土资源航空物探遥感中心海洋地质与地球物理业务管理流程开发中标公告.html
808 2018.04.24中国国土资源

1577 2018.06.19中央电视台数据中心资源池相关设备全包代维中标公告.html
1578 2018.06.19中央财经大学保安服务采购项目中标公告.html
1579 2018.06.19中央音乐学院票务服务外包项目中标公告.html
1580 2018.06.19公安部机关政府采购办公室公安部全国刑专系统623子系统大数据平台项目中标公告.html
1581 2018.06.19公安部机关政府采购办公室公安部全国刑专系统623子系统应用软件项目中标公告.html
1582 2018.06.19北京化工大学校园网出口带宽光纤专线2Gbps接入项目中标公告.html
1583 2018.06.19北京化工大学校园网出口带宽光纤专线3.5Gbps接入项目中标公告.html
1584 2018.06.19国家体育总局财务管理和审计中心国家体育总局财务管理信息系统开发项目中标公告.html
1585 2018.06.19国家气象信息中心多传感器遥感遥测数据一体化实时分析系统研发中标公告.html
1586 2018.06.19国家知识产权局专利局专利审查协作天津中心永久办公用房项目会议系统音视频设计和集成中标公告.html
1587 2018.06.19国家税务总局湖北省税务局实名办税系统日常运维服务采购项目中标公告.html
1588 2018.06.19国家税务总局湖北省税务局微平台2018年纳税人端优化开发服务采购项目中标公告.html
1589 2018.06.19国家认证认可监督管理委员会检验检测机构资质认定信息化监管采购项目中标公告.html
1590 2018.06.19委机关及所属单位内部审计监督与评审评价项目中标公告.html
1591 2018.06.19铁路公安局铁路公安民警退休纪念章制作项目公开招标中标公告.html
1592 2018.06.20中南大学2018-2021年度固定资产统一保险服务采购项目中标公告.html
1593 2018.06.20中国农业大学教学区保安服务采购项目中标公告.html
1594 2018.06.20中国国际贸易促进委员会2018年贸促会境外展览承办单位招标采购项目-中国（拉脱维亚）机电品牌展览会中标公告.html
1595 2018.06.20中国地质调查局成都地质调查中心2018年度四川盆地龙马溪组页

2375 2018.08.13最高人民法院司法行政装备管理局国家司法审判信息资源管理和服务平台项目（重新招标）中标公告.html
2376 2018.08.13民政部委托第三方机构开展行政区划调整效果评估采购项目中标公告.html
2377 2018.08.13浙江大学海宁国际校区2018-2020年度修缮定点供应商中标公告.html
2378 2018.08.13重庆市税务干部学校数字化校园中标公告.html
2379 2018.08.14上海国际招标有限公司关于华东师范大学中山北路校区保安服务项目（项目编号：18405701）的中标公告.html
2380 2018.08.14中国科学院计算机网络信息中心ARP系统网站群平台灾备能力扩充采购项目中标公告.html
2381 2018.08.14全国组织机构统一社会信用代码数据服务中心国家法人单位信息资源库（一期）项目（质检总局部分）信息化监理中标公告.html
2382 2018.08.14北京大学人民医院2018年护理员服务项目中标公告.html
2383 2018.08.14国家体育总局体育彩票管理中心制定《体育彩票机构执行实施细则》采购项目中标公告.html
2384 2018.08.14国家体育总局体育彩票管理中心用户画像标签体系及应用模型建设项目中标公告.html
2385 2018.08.14国家气象信息中心气象大数据汇交及融合应用系统中标公告.html
2386 2018.08.14国家税务局总局贵州省税务局网络安全运维服务及核心数据库安全管控服务采购项目（包1：网络安全运维服务二次招标）中标公告.html
2387 2018.08.14国家税务总局2018年税务干部综合管理信息系统机构改革软件服务项目中标公告.html
2388 2018.08.14黄河水利委员会河南水文水资源局黄委大江大河水文监测系统建设工程（一期）水文测验船舶设计中标公告.html
2389 2018.08.15《世界军事》《中国军队》期刊印刷服务项目中标公告.html
2390 2018.08.15东华大学松江校区中央空调设备运行日常管理、维护保养服务中标公告.html
2391 2018.08.15中国人民解放军95997部队餐饮社会化保障项目中标公告.html
2392 2018.08.15中国农业大学新楼物业服务采购项

3166 2018.10.192018年中国浦东干部学院消防报警系统升级改造工程项目管理及监理服务项目中标公告.html
3167 2018.10.19中国地质调查局天津地质调查中心招平金矿带地震剖面测量中标公告.html
3168 2018.10.19中国民用航空西北地区空中交通管理局民航西北空管局新建塔台电子进程单备份工程系统集成项目中标公告.html
3169 2018.10.19中国疾病预防控制中心妇幼保健中心妇幼健康信息化建设项目中标公告.html
3170 2018.10.19中国科学技术协会学会学术部全国学会组织管理信息平台升级完善项目中标公告.html
3171 2018.10.19中央国家机关2018-2020年工程监理定点企业招标项目中标公告.html
3172 2018.10.19中山大学总务处维修工程监理服务项目中标公告.html
3173 2018.10.19国家气象中心海洋气象多源资料区域快速循环同化子系统（重新招标）中标公告.html
3174 2018.10.19国家税务总局宁夏回族自治区税务局税收业务专网线路租赁服务采购项目中标公告.html
3175 2018.10.19广州海洋地质调查局2018年度船舶修理项目的中标结果调整公告（包组一、包组二）.html
3176 2018.10.22中华人民共和国塘沽出入境边防检查站物业服务项目中标公告.html
3177 2018.10.22中国传媒大学校园安全综合管理平台采购中标公告.html
3178 2018.10.22中国信息通信研究院工业互联网标识解析国家顶级节点建设（一期）北京节点部分基础硬件采购项目中标公告.html
3179 2018.10.22中国信息通信研究院离散制造业信息物理系统关键共性技术测试验证能力提升数字化模型转换测试工具项目中标公告.html
3180 2018.10.22中国新闻出版研究院新闻出版大数据应用工程——国家知识服务平台项目——内容开放服务系统中标公告.html
3181 2018.10.22中国海洋大学流程协同中心系统采购项目中标公告.html
3182 2018.10.22南开大学商学院物业管理服务项目中标公告.html
3183 2018.10.22中国科协创新战略研究院采集工程馆藏基地数字存储扩容中标公告.html
3184 2018.10

...

11095 2020.05.11浙江大学修缮工程全过程施工监理服务（2020-2023年）中标公告.html
11096 2020.05.11浙江大学精品在线开放课程建设及混合式教学运行服务中标公告.html
11097 2020.05.11陆军装备部驻北京地区军事代表局驻天津地区军事代表室营区物业管理社会化保障项目中标公告.html
11098 2020.05.14中华人民共和国吉林出入境边防检查总站选定信息化项目监理服务公司采购项目中标公告.html
11099 2020.05.14中国国家博物馆2020-2022年度保安外包服务采购项目中标公告.html
11100 2020.05.14中国教育电视台2020—2021年度信息安全服务项目中标公告.html
11101 2020.05.14中国地质科学院矿产资源研究所松潘－甘孜成锂带锂铍多金属大型资源基地综合调查评价2020年度钻探工程中标公告.html
11102 2020.05.14中国地质调查局南京地质调查中心2020年地质调查项目委托业务（2标段）中标公告.html
11103 2020.05.14中国地质调查局成都地质调查中心四川广安资源环境综合地质调查委托业务中标公告.html
11104 2020.05.14中国地质调查局成都地质调查中心四川广安资源环境综合地质调查项目广安城市规划区工程地质钻探（第二次）中标公告.html
11105 2020.05.14中国教育电视台2020年度全台网络系统及新媒体平台硬件维保服务项目中标公告.html
11106 2020.05.14中国民航大学老校区回顾性环境影响评价项目中标公告.html
11107 2020.05.14中国浦东干部学院2020年学员保险服务招标中标公告.html
11108 2020.05.14中国环境科学研究院青岛市大气污染深度解析与治理攻关研究项目--大气源清单污染特征及模型构建应用研究中标公告.html
11109 2020.05.14中央音乐学院AI超算基础平台建设项目中标公告.html
11110 2020.05.14交通运输部公路局国家重点公路建设项目2020年度初步设计审批技术咨询服务业绩审查及初步设计审批技术咨询服务（2020年度第一批）中标公告.html
11111 2020.05.14北京化工大学园林绿化养护服务采购项

11915 2020.06.28国家无线电监测中心短波TDOA监测定位子系统项目中标公告.html
11916 2020.06.28国家无线电监测中心短波监测一体化平台项目中标公告.html
11917 2020.06.28国家林业和草原局信息中心2020年国家林业和草原局信息系统运行维护服务项目中标公告.html
11918 2020.06.28国家气象信息中心国家级气象雷达业务安全运营中心建设项目中标公告.html
11919 2020.06.28国家移民管理局本式及单页证件中标公告.html
11920 2020.06.28国家税务总局江西省税务局业务专网线路续费及升级采购项目（主备中心互联链路包）中标公告.html
11921 2020.06.28国家税务总局江西省税务局业务专网线路续费及升级采购项目（主链路包）中标公告.html
11922 2020.06.28国家税务总局江西省税务局业务专网线路续费及升级采购项目（备份链路包）中标公告.html
11923 2020.06.28国家税务总局浙江省税务局ITS纳税人端技术支持咨询服务项目中标公告.html
11924 2020.06.28国家税务总局漳州市税务局物业管理服务项目中标公告.html
11925 2020.06.28国家税务总局荆门市税务局机关、第一税务分局、第二税务分局食堂劳务外包服务中标公告.html
11926 2020.06.28复旦大学附属中山医院融合5G的医联体影像协同创新云平台中标公告.html
11927 2020.06.28森林消防局大庆航空救援支队2020年大庆航空救援支队智能管理平台建设项目中标公告.html
11928 2020.06.28森林消防局大庆航空救援支队2020年大庆航空救援支队航材封存服务项目（第二次招标）中标公告.html
11929 2020.06.28水利部信息中心三峡库区水域监管部级视频级联集控系统整合升级项目——水利视频级联集控平台开发中标公告.html
11930 2020.06.28海关总署物资装备采购中心海关总署2020年电子口岸数据中心智能密码钥匙采购项目中标公告.html
11931 2020.06.28西城区消防救援支队大屋脊筒子楼简易喷淋设备采购项目-监理中标公告.html
11932 2020.06.28西安交通大学后勤保障部创新港

12729 2020.07.31四川省公安厅特勤局食堂原材料配送项目中标公告.html
12730 2020.07.31国家工业信息安全发展研究中心工业互联网攻防测试联合管控系统基础资源建设项目中标公告.html
12731 2020.07.31国家工业信息安全发展研究中心智能制造行业工业互联网企业级集中化安全监测平台综合测试中标公告.html
12732 2020.07.31国家市场监督管理总局产品质量安全风险监测站建设工作项目（第一批）中标公告.html
12733 2020.07.31国家税务总局河南省税务局外网网站平台改造项目中标公告.html
12734 2020.07.31应急管理部消防救援局消防员信息系统项目中标公告.html
12735 2020.07.31教育部直属单位财务共享平台财务信息系统建设项目公开招标中标公告.html
12736 2020.07.31清华大学2020年专项审计采购项目中标公告.html
12737 2020.07.31民政部信息中心金民工程一期民政政务监管系统建设项目中标公告.html
12738 2020.07.31水利部信息中心河湖治理保护本底数据遥感调查项目遥感基础数据处理和入库服务中标公告.html
12739 2020.07.31清华大学领导干部任期经济责任审计服务采购项目中标公告.html
12740 2020.07.31生态环境部政府网站及网站群技术运维项目中标公告.html
12741 2020.07.31生态环境部流域生态环境监督管理局自动化办公系统项目中标公告.html
12742 2020.07.31西南交通大学九里校区校园综合治理监理中标公告.html
12743 2020.08.01中国科学院南海海洋研究所“实验3”科考船特检修理采购项目中标公告.html
12744 2020.08.03中华人民共和国吉林出入境边防检查总站选定油料供应商采购项目中标公告.html
12745 2020.08.03中南大学教学发展中心创新空间平台建设项目中标公告.html
12746 2020.08.03中国信息通信研究院工业互联网平台组件与设备检测认证服务平台（二期）购置项目-工业互联网5G连接模块测试系统采购项目中标公告.html
12747 2020.08.03中国信息通信研究院物联网核心设备认证平台（泰尔终

13495 2020.09.21商务部电子商务和信息化司信息化项目监督与管理中标公告.html
13496 2020.09.21国家市场监督管理总局信息中心2020年市场监管总局安全设施运维及维保服务项目中标公告.html
13497 2020.09.21国家林业和草原局信息中心2020年国家林业和草原局网络及视频会议等运维服务项目中标公告.html
13498 2020.09.21国家税务总局天津市南开区税务局机房升级建设项目中标公告.html
13499 2020.09.21国家税务总局广西壮族自治区税务局电子税务局优化改造及对接广西政务一体化平台项目（项目编号：GXZC2020-G3-00090-JHGL）中标公告.html
13500 2020.09.21国家能源局信息中心办公用房租赁项目中标公告.html
13501 2020.09.21复旦大学附属肿瘤医院智慧医院人工智能创新应用中标公告.html
13502 2020.09.21天津市红桥区消防救援支队车辆维修保养项目中标公告.html
13503 2020.09.21天津海关缉私局缉私业务大楼物业服务项目（项目编号：0615-204120050215）中标结果公告.html
13504 2020.09.21文艺资源内容智能传播与运营融合服务平台建设项目中标公告.html
13505 2020.09.21民政部机关办公区物业服务项目中标（成交）结果公告.html
13506 2020.09.21气象信息化系统工程—气象基础数据质量管理建设任务（一期）中标公告.html
13507 2020.09.21陕西省地震烈度速报与预警系统省级数据处理中心集成安装采购项目中标公告.html
13508 2020.09.22中国人民大学教师学术精品海外推广项目中标公告.html
13509 2020.09.22中国医学科学院医学信息研究所医学健康信息舆情平台构建项目中标公告.html
13510 2020.09.22中国国际贸易促进委员会2020年贸促会数字展览承办单位招标采购项目-中国—中东及北非（摩洛哥）国际贸易数字展览会中标公告.html
13511 2020.09.22中国国际贸易促进委员会2020年贸促会数字展览承办单位招标采购项目-中国—中美洲及南太平洋地区国际贸易数字展览会中标公告.html
13512

14302 2020.11.06中国科协信息中心2020年“一体两翼”官网建设运营项目（重招）中标公告.html
14303 2020.11.06中国科学技术大学中国教育科研网宽带接入中标公告.html
14304 2020.11.06中国科学院上海高等研究院基于境内工艺的定制电路增强设计——SRAM,RF-I中标公告.html
14305 2020.11.06中国科学院上海高等研究院基于境内工艺的定制电路增强设计——标准单元库,Efuse中标公告.html
14306 2020.11.06中国科学院上海高等研究院基于境内工艺的定制电路增强设计——高速I,O中标公告.html
14307 2020.11.06中国科学院上海高等研究院增强标准单元库定制电路设计中标公告.html
14308 2020.11.06中国科学院上海高等研究院大规模用例生成系统顶层架构需求中标公告.html
14309 2020.11.06中国科学院上海高等研究院片间高速互联总线关键技术研究和验证中标公告.html
14310 2020.11.06中国科学院上海高等研究院芯片试流片及封测——B版中标公告.html
14311 2020.11.06中国科学院上海高等研究院面向境内工艺的FSB设计中标公告.html
14312 2020.11.06中国科学院上海高等研究院面向境内工艺的PLL,Thermal设计中标公告.html
14313 2020.11.06中国科学院烟台海岸带研究所物业服务中标公告.html
14314 2020.11.06中国科学院空天信息创新研究院地球大数据GPU集群计算系统采购项目中标公告.html
14315 2020.11.06中国经济信息社有限公司新华社金融信息平台一期银行信贷风控辅助系统中标公告.html
14316 2020.11.06中国证券业协会证券业从业人员资格考试计算机化考试服务项目中标公告.html
14317 2020.11.06交通运输部路网监测与应急处置中心高速公路收费系统改造项目部级工程第10标段第三方软件测评及信息资源目录编制服务中标公告.html
14318 2020.11.06交通运输部路网监测与应急处置中心高速公路收费系统改造项目部级工程第11标段等保测试及商用密码安全性评估服务中标公告.html
14319 2020.11.06北京

In [31]:
bidder_result_table = html_removing_whiteSpaceString.xpath('//tbody/tr/td/text()') 

In [32]:
bidder_result_table

['序号',
 '中标供应商名称',
 '中标供应商联系地址',
 '中标金额(万元)',
 '1',
 '广州医药有限公司',
 '广东省广州市大同路103号',
 '0.000000']

In [37]:
len(html_removing_whiteSpaceString.xpath('//tbody/tr/td')) # table data 单元格对象个数

8

In [38]:
len(html_removing_whiteSpaceString.xpath('//tbody/tr')) # table row 对象个数

2

In [33]:
f

'<!doctype html>\n<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<meta http-equiv="pragma" content="no-cache">\n<meta http-equiv="cache-control" content="no-cache">\n<meta http-equiv="expires" content="0">\n<title>2017年粤北片区监狱医院药品、医用耗材及检验试剂集中采购配送商遴选（重招）中标公告</title>\n<link href="/css/detail.css" rel="stylesheet" type="text/css" />\n<script language="javascript" src="/js/jquery-3.2.1.min.js"></script>\n<script language="javascript" src="http://pub.ccgp.gov.cn/common/js/jquery.qrcode.min.js"></script>\n</head>\n<body id="detail">\n<link href="http://www.ccgp.gov.cn/css/inc.css" rel="stylesheet" type="text/css" />\n<div class="v4incheadertop">\n    <div class="v4incheadertop_tel_block">\n        <div class="v4incheadertop_tel">\n            <p class="cl">财政部唯一指定政府采购信息网络发布媒体 国家级政府采购专业网站</p><p class="cr">服务热线：400-810-1996</p>\n        </div>\n    </div>\n    <div class="v4incheadertop_logosearch_block2">\n        <div class="v4incheadertop_logosearch">\